# User input

In [1]:
start_year_past = 2021
stop_year_past = 2040
start_year_future = 2041
stop_year_future = 2060

# Import package

In [2]:
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import gumbel_r
from scipy.stats import gumbel_l
import os
import os.path
import math

from Functions_Indicators_precipitation import changes_in_indicators

# Import data

In [3]:
out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6'
csv_path = os.path.join(out_path,'csv_file','pr_day_2021-2060','All_projects_moz_precipitation_2021-2060_without_month_year.csv')
precipitation_2021_2060 = pd.read_csv(csv_path,header=[0,1],index_col=[0,1,2,3,4])*86400 # units mm/day :  1 kg/m2/s = 86400 mm/day
# source: https://www.researchgate.net/post/How-do-I-convert-ERA-Interim-precipitation-estimates-from-kg-m2-s-to-mm-day
list_time_2021_2060=pd.date_range('01-01-2021','31-12-2060', freq='D').strftime('%d-%m-%Y').values.tolist()

In [4]:
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True) # make a coåy that can be modified without changin the original

In [5]:
precipitation_2021_2060_copy

Longitude  \
                                                                    33.625   
Name project          Experiment Model      Date       Latitude              
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125    5.057179   
                                            02-01-2021 -16.125   17.624178   
                                            03-01-2021 -16.125    7.632648   
                                            04-01-2021 -16.125    4.736492   
                                            05-01-2021 -16.125    4.981353   
...                                                                    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125         NaN   
                                            28-12-2060 -13.125         NaN   
                                            29-12-2060 -13.125         NaN   
                                            30-12-2060 -13.125         NaN   
                                            31-12-2060 -13.125         NaN   

                                                                               \
                                                                39.875 36.875   
Name project          Experiment Model      Date       Latitude                 
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125     NaN    NaN   
                                            02-01-2021 -16.125     NaN    NaN   
                                            03-01-2021 -16.125     NaN    NaN   
                                            04-01-2021 -16.125     NaN    NaN   
                                            05-01-2021 -16.125     NaN    NaN   
...                                                                ...    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125     NaN    NaN   
                                            28-12-2060 -13.125     NaN    NaN   
                                            29-12-2060 -13.125     NaN    NaN   
                                            30-12-2060 -13.125     NaN    NaN   
                                            31-12-2060 -13.125     NaN    NaN   

                                                                           
                                                                   40.375  
Name project          Experiment Model      Date       Latitude            
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125        NaN  
                                            02-01-2021 -16.125        NaN  
                                            03-01-2021 -16.125        NaN  
                                            04-01-2021 -16.125        NaN  
                                            05-01-2021 -16.125        NaN  
...                                                                   ...  
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125   0.000000  
                                            28-12-2060 -13.125   0.435942  
                                            29-12-2060 -13.125   6.300328  
                                            30-12-2060 -13.125   8.168522  
                                            31-12-2060 -13.125   0.737792  

[6136200 rows x 4 columns]

In [6]:
# delete model 'NESM3' that does not have any value
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True)
precipitation_2021_2060_copy.drop('NESM3', level=2, axis=0, inplace=True)
precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]

C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_182792\1858925233.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]


KeyError: ('Chimoio_2ML_Reservoir', 'ssp126', 'NESM3')

In [7]:
# treatment of dataframe to make useable
df_years=precipitation_2021_2060_copy.copy(deep=True)
#df_year = df_year1.loc[('PT_Revubue_2_Rev_2_01')] # select one project
df_years.columns = df_years.columns.droplevel(0) # drop first level of column name
df_years['33.625'] = df_years['33.625'].fillna(0)
df_years['39.875'] = df_years['39.875'].fillna(0)
df_years['36.875'] = df_years['36.875'].fillna(0)
df_years['40.375'] = df_years['40.375'].fillna(0)
df_years['33.625'] = df_years['33.625'] + df_years['39.875'] + df_years['36.875'] + df_years['40.375']
df_years = df_years[['33.625']]
df_years=df_years.rename(columns={df_years.columns[0]:'Precipitation mm/day period 2021-2060'})
df_years=df_years.droplevel(level=4) # drop latitude index
df_years=df_years.reset_index()

Year = df_years[['Date']].values.reshape(len(df_years[['Date']].values),)
for i in np.arange(0,len(df_years[['Date']].values)):
    Year[i]=int(Year[i][6:10])

df_years['Year'] = Year
df_years

,Name project,Experiment,Model,Date,Precipitation mm/day period 2021-2060,Year
0,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2021,5.057179,2021
1,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2021,17.624178,2021
2,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2021,7.632648,2021
3,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2021,4.736492,2021
4,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2021,4.981353,2021
...,...,...,...,...,...,...
5727115,PT__Dumping_Site,ssp370,TaiESM1,27-12-2060,0.000000,2060
5727116,PT__Dumping_Site,ssp370,TaiESM1,28-12-2060,0.435942,2060
5727117,PT__Dumping_Site,ssp370,TaiESM1,29-12-2060,6.300328,2060
5727118,PT__Dumping_Site,ssp370,TaiESM1,30-12-2060,8.168522,2060


In [8]:
# select past datas, between start_year_future and stop_year_future
df_past = df_years.loc[df_years['Year'].between(start_year_past,stop_year_past)]
df_past

,Name project,Experiment,Model,Date,Precipitation mm/day period 2021-2060,Year
0,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2021,5.057179,2021
1,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2021,17.624178,2021
2,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2021,7.632648,2021
3,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2021,4.736492,2021
4,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2021,4.981353,2021
...,...,...,...,...,...,...
5719810,PT__Dumping_Site,ssp370,TaiESM1,27-12-2040,0.184525,2040
5719811,PT__Dumping_Site,ssp370,TaiESM1,28-12-2040,5.674063,2040
5719812,PT__Dumping_Site,ssp370,TaiESM1,29-12-2040,0.471744,2040
5719813,PT__Dumping_Site,ssp370,TaiESM1,30-12-2040,5.973260,2040


In [9]:
# select future data, between start_year_future and stop_year_future
df_future = df_years.loc[df_years['Year'].between(start_year_future,stop_year_future)]
df_future

,Name project,Experiment,Model,Date,Precipitation mm/day period 2021-2060,Year
7305,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2041,12.308317,2041
7306,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2041,8.492493,2041
7307,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2041,10.244099,2041
7308,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2041,30.440408,2041
7309,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2041,23.266100,2041
...,...,...,...,...,...,...
5727115,PT__Dumping_Site,ssp370,TaiESM1,27-12-2060,0.000000,2060
5727116,PT__Dumping_Site,ssp370,TaiESM1,28-12-2060,0.435942,2060
5727117,PT__Dumping_Site,ssp370,TaiESM1,29-12-2060,6.300328,2060
5727118,PT__Dumping_Site,ssp370,TaiESM1,30-12-2060,8.168522,2060


## Average yearly precipitation

In [10]:
from Functions_Indicators_precipitation import yearly_avg_pr

In [11]:
# For historical period
title_column = 'Yearly average precipitation mm/year period '+str(start_year_past)+'-'+str(stop_year_past)
df_years_avg_past_distribution= yearly_avg_pr(df_past,title_column)

In [12]:
# For future period
title_column = 'Yearly average precipitation mm/year period '+str(start_year_future)+'-'+str(stop_year_future)
df_years_avg_future_distribution= yearly_avg_pr(df_future,title_column)

In [13]:
# Comparaison between past and future period

In [14]:
changes_past_future_indicator = pd.DataFrame()
changes_past_future_indicator1 = changes_in_indicators(df_years_avg_past_distribution,df_years_avg_future_distribution,'Average yearly precipitation','Incremental rainfall change')
changes_past_future_indicator=pd.concat([changes_past_future_indicator,changes_past_future_indicator1])
changes_past_future_indicator

Incremental rainfall change  \
                           Average yearly precipitation   
                              Change in the median in %   
Chimoio_2ML_Reservoir                         -2.202097   
Mugica_dam_road                               -3.949296   
PT1_Pemba__Breidge_Paquite                    -2.045428   
PT_Nicoadala_F5                               -3.107802   
PT_Revubue_2_Rev_2_01                          1.082005   
PT__Dumping_Site                              -2.344221   
WTP_gate_cnr1                                  1.082005   

                                                         \
                                                          
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                         -1.747428   
Mugica_dam_road                               -3.806494   
PT1_Pemba__Breidge_Paquite                    -1.154481   
PT_Nicoadala_F5                               -6.790416   
PT_Revubue_2_Rev_2_01                         -1.444476   
PT__Dumping_Site                              -1.188254   
WTP_gate_cnr1                                 -1.444476   

                                                         
                                                         
                           Change in 90-th percentile %  
Chimoio_2ML_Reservoir                          0.562997  
Mugica_dam_road                               -2.197210  
PT1_Pemba__Breidge_Paquite                    -2.357272  
PT_Nicoadala_F5                               -1.974410  
PT_Revubue_2_Rev_2_01                          0.881545  
PT__Dumping_Site                              -2.925441  
WTP_gate_cnr1                                  0.881545

## Seasonal average precipitation

In [15]:
from Functions_Indicators_precipitation import avg_dry_season_precipitation

In mozambique, dry season from April to september. Humid season from October to March

In [16]:
# historic

In [17]:
title_column1 = 'Yearly average precipitation mm/season per year for the period '+str(start_year_past)+'-'+str(stop_year_past)
pr_past_dry_season_mean_distribution = avg_dry_season_precipitation(df_past,title_column1)

C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:392: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pr_dry_season_mean_distribution=pr_dry_season_mean_distribution.reset_index().drop('Season',axis=1).set_index('Name project')


In [18]:
# future

In [19]:
title_column1 = 'Yearly average precipitation mm/season per year for the period '+str(start_year_future)+'-'+str(stop_year_future)
pr_future_dry_season_mean_distribution = avg_dry_season_precipitation(df_future,title_column1)

C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:392: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pr_dry_season_mean_distribution=pr_dry_season_mean_distribution.reset_index().drop('Season',axis=1).set_index('Name project')


In [20]:
pr_future_dry_season_mean_distribution

Yearly average precipitation mm/season per year for the period 2041-2060  \
                                                                                              count   
Name project                                                                                          
Chimoio_2ML_Reservoir                                                    56.0                         
Mugica_dam_road                                                          56.0                         
PT1_Pemba__Breidge_Paquite                                               56.0                         
PT_Nicoadala_F5                                                          56.0                         
PT_Revubue_2_Rev_2_01                                                    56.0                         
PT__Dumping_Site                                                         56.0                         
WTP_gate_cnr1                                                            56.0                         

                                                                           \
                                  mean        std         min         10%   
Name project                                                                
Chimoio_2ML_Reservoir        76.069421  12.852272   40.298488   59.005105   
Mugica_dam_road             148.783259  19.809934  107.704550  120.591997   
PT1_Pemba__Breidge_Paquite  176.540563  24.675154  112.545617  145.813778   
PT_Nicoadala_F5             315.486927  39.395620  253.270941  262.127148   
PT_Revubue_2_Rev_2_01        25.702344   4.949114   13.166689   19.973013   
PT__Dumping_Site            163.924495  23.033458  104.224709  133.433776   
WTP_gate_cnr1                25.702344   4.949114   13.166689   19.973013   

                                                                
                                   50%         90%         max  
Name project                                                    
Chimoio_2ML_Reservoir        77.393657   91.692715  101.685919  
Mugica_dam_road             149.160700  172.969215  187.026991  
PT1_Pemba__Breidge_Paquite  174.456970  215.753152  230.877777  
PT_Nicoadala_F5             313.436397  367.314841  416.791587  
PT_Revubue_2_Rev_2_01        25.917032   31.110132   40.680932  
PT__Dumping_Site            162.571363  199.081215  211.812041  
WTP_gate_cnr1                25.917032   31.110132   40.680932

In [21]:
# Comparaison between past and future period

In [22]:
changes_past_future_indicator1 = changes_in_indicators(pr_past_dry_season_mean_distribution,pr_future_dry_season_mean_distribution,'Average dry season precipitation','Incremental rainfall change')
changes_past_future_indicator=pd.merge(changes_past_future_indicator, changes_past_future_indicator1, left_index=True, right_index=True, how='outer')

In [23]:
changes_past_future_indicator

Incremental rainfall change  \
                           Average yearly precipitation   
                              Change in the median in %   
Chimoio_2ML_Reservoir                         -2.202097   
Mugica_dam_road                               -3.949296   
PT1_Pemba__Breidge_Paquite                    -2.045428   
PT_Nicoadala_F5                               -3.107802   
PT_Revubue_2_Rev_2_01                          1.082005   
PT__Dumping_Site                              -2.344221   
WTP_gate_cnr1                                  1.082005   

                                                         \
                                                          
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                         -1.747428   
Mugica_dam_road                               -3.806494   
PT1_Pemba__Breidge_Paquite                    -1.154481   
PT_Nicoadala_F5                               -6.790416   
PT_Revubue_2_Rev_2_01                         -1.444476   
PT__Dumping_Site                              -1.188254   
WTP_gate_cnr1                                 -1.444476   

                                                         \
                                                          
                           Change in 90-th percentile %   
Chimoio_2ML_Reservoir                          0.562997   
Mugica_dam_road                               -2.197210   
PT1_Pemba__Breidge_Paquite                    -2.357272   
PT_Nicoadala_F5                               -1.974410   
PT_Revubue_2_Rev_2_01                          0.881545   
PT__Dumping_Site                              -2.925441   
WTP_gate_cnr1                                  0.881545   

                                                             \
                           Average dry season precipitation   
                                  Change in the median in %   
Chimoio_2ML_Reservoir                              1.126932   
Mugica_dam_road                                   -5.900586   
PT1_Pemba__Breidge_Paquite                        -3.362560   
PT_Nicoadala_F5                                   -4.477743   
PT_Revubue_2_Rev_2_01                             -1.507191   
PT__Dumping_Site                                  -3.542630   
WTP_gate_cnr1                                     -1.507191   

                                                         \
                                                          
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                        -11.658801   
Mugica_dam_road                              -13.040792   
PT1_Pemba__Breidge_Paquite                    -8.509459   
PT_Nicoadala_F5                               -9.723633   
PT_Revubue_2_Rev_2_01                         -1.211725   
PT__Dumping_Site                              -9.968265   
WTP_gate_cnr1                                 -1.211725   

                                                         
                                                         
                           Change in 90-th percentile %  
Chimoio_2ML_Reservoir                          4.855074  
Mugica_dam_road                               -4.254136  
PT1_Pemba__Breidge_Paquite                    -2.097405  
PT_Nicoadala_F5                                0.382411  
PT_Revubue_2_Rev_2_01                         -7.601198  
PT__Dumping_Site                              -3.117568  
WTP_gate_cnr1                                 -7.601198

## Return period

In [ ]:
 #INDICATOR NOT FINISHED
#from Functions_Indicators_precipitation import threshold_coresponding_to_return_period
from Functions_Indicators_precipitation import dataframe_threshold_coresponding_to_return_period
#from Functions_Indicators_precipitation import dataframe_future_return_period_of_1_day_event

In [ ]:
def return_period_coresponding_to_threshold(Z):
    (loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
    # gumbel_r is chosen because
    #try:
    p_non_exceedance = gumbel_r.cdf(max(Z),loc,scale)
    #except OverflowError: # the result is not finite
        
   #     if math.isinf(gumbel_r.cdf(threshold,loc,scale)) and gumbel_r.cdf(max(Z),loc,scale)<0:
   #         # the result is -inf
    #        threshold_coresponding = 0 # the value of wero is imposed
    return_period_coresponding = 1/(1-p_non_exceedance)
    return round(return_period_coresponding)

In [ ]:
def dataframe_future_return_period_of_1_day_event(df):
    df_copy=df.copy(deep=True)
    df_copy=df_copy.drop(labels='Date',axis=1)
    df_max = df_copy.groupby(['Name project','Experiment','Model','Year']).max() # maximum    
    midx = pd.MultiIndex.from_product([list(set(df_copy[df_copy.columns[0]])),list(set(df_copy[df_copy.columns[1]])),list(set(df_copy[df_copy.columns[2]]))],names=['Name project','Experiment', 'Model'])
    cols = ['Return period 1 day event']
    return_period = pd.DataFrame(data = [], 
                                index = midx,
                                columns = cols)
    for name_p in return_period.index.levels[0].tolist():
        for ssp in return_period.index.levels[1].tolist():
            for model in return_period.index.levels[2].tolist():
                print('Name project '+name_p+ ' ssp '+ssp+ ' model '+model)
                Z=df_max.loc[(name_p,ssp,model)].values.reshape(len(df_max.index.levels[3]),)
                return_period.loc[(name_p,ssp,model),('Return period 1 day event')] = return_period_coresponding_to_threshold(Z)
    return return_period

In [ ]:
# past

In [ ]:
return_period_past=dataframe_threshold_coresponding_to_return_period(df_past)
return_period_past

In [ ]:
# dataframe contains statistics about past value for return period
return_period_distribution_past=return_period_past.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9]) # astype(int) necessary to have stats min, max and percentiles
return_period_distribution_past

In [ ]:
# future

In [ ]:
return_period_1_day_event_future=dataframe_future_return_period_of_1_day_event(df_future)
return_period_1_day_event_future

In [ ]:
df_copy=df_future.copy(deep=True)
df_copy=df_copy.drop(labels='Date',axis=1)
Z=df_max.loc[('Chimoio_2ML_Reservoir','ssp126','ACCESS-CM2')].values.reshape(len(df_max.index.levels[3]),)
(loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
# gumbel_r is chosen because
#try:
p_non_exceedance = gumbel_r.cdf(max(Z),loc,scale)
p_non_exceedance
#except OverflowError: # the result is not finite

#     if math.isinf(gumbel_r.cdf(threshold,loc,scale)) and gumbel_r.cdf(max(Z),loc,scale)<0:
#         # the result is -inf
#        threshold_coresponding = 0 # the value of wero is imposed
#return_period_coresponding = 1/(1-p_non_exceedance)

In [ ]:
# dataframe contains statistics about futur value for return period
return_period_distribution_future=return_period_1_day_event_future.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9]) # astype(int) necessary to have stats min, max and percentiles
return_period_distribution_future

In [ ]:
# Comparaison between past and future period
changes_past_future_indicator1 = changes_in_indicators(return_period_distribution_past,return_period_distribution_future,'Future return period of one day with a current return period of 100 years','Extreme rainfall change')
changes_past_future_indicator=pd.merge(changes_past_future_indicator, changes_past_future_indicator1, left_index=True, right_index=True, how='outer')
changes_past_future_indicator

## N-day event

### 1-day event

In [24]:
from Functions_Indicators_precipitation import dataframe_1_day_event

In [25]:
# past
df_1_day_event_past = dataframe_1_day_event(df_past)
df_1_day_event_past

Maximum 1 day rainfall mm 2021-2040
Name project          Experiment Model      Year                                     
Chimoio_2ML_Reservoir ssp126     ACCESS-CM2 2021                            22.153287
                                            2022                            45.533448
                                            2023                            23.700950
                                            2024                            32.253806
                                            2025                            21.849809
...                                                                               ...
WTP_gate_cnr1         ssp585     TaiESM1    2036                            52.333608
                                            2037                            38.781290
                                            2038                            49.618615
                                            2039                            51.240096
                                            2040                            19.995862

[7840 rows x 1 columns]

In [26]:
# past
df_1_day_event_future = dataframe_1_day_event(df_future)
df_1_day_event_future

Maximum 1 day rainfall mm 2041-2060
Name project          Experiment Model      Year                                     
Chimoio_2ML_Reservoir ssp126     ACCESS-CM2 2041                            26.993741
                                            2042                            34.838782
                                            2043                            28.120784
                                            2044                            31.804512
                                            2045                            23.991481
...                                                                               ...
WTP_gate_cnr1         ssp585     TaiESM1    2056                            50.823568
                                            2057                            33.281169
                                            2058                            18.805485
                                            2059                            19.757411
                                            2060                            20.890592

[7840 rows x 1 columns]

In [27]:
df_1_day_event_past_distribution = df_1_day_event_past.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9]) # astype(int) necessary to have stats min, max and percentiles
df_1_day_event_future_distribution = df_1_day_event_future.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9]) # astype(int) necessary to have stats min, max and percentiles

In [28]:
# Comparaison between past and future period
changes_past_future_indicator1 = changes_in_indicators(df_1_day_event_past_distribution,df_1_day_event_future_distribution,'Maximum one day rainfall','Extreme rainfall change')
changes_past_future_indicator=pd.merge(changes_past_future_indicator, changes_past_future_indicator1, left_index=True, right_index=True, how='outer')
changes_past_future_indicator

Incremental rainfall change  \
                           Average yearly precipitation   
                              Change in the median in %   
Chimoio_2ML_Reservoir                         -2.202097   
Mugica_dam_road                               -3.949296   
PT1_Pemba__Breidge_Paquite                    -2.045428   
PT_Nicoadala_F5                               -3.107802   
PT_Revubue_2_Rev_2_01                          1.082005   
PT__Dumping_Site                              -2.344221   
WTP_gate_cnr1                                  1.082005   

                                                         \
                                                          
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                         -1.747428   
Mugica_dam_road                               -3.806494   
PT1_Pemba__Breidge_Paquite                    -1.154481   
PT_Nicoadala_F5                               -6.790416   
PT_Revubue_2_Rev_2_01                         -1.444476   
PT__Dumping_Site                              -1.188254   
WTP_gate_cnr1                                 -1.444476   

                                                         \
                                                          
                           Change in 90-th percentile %   
Chimoio_2ML_Reservoir                          0.562997   
Mugica_dam_road                               -2.197210   
PT1_Pemba__Breidge_Paquite                    -2.357272   
PT_Nicoadala_F5                               -1.974410   
PT_Revubue_2_Rev_2_01                          0.881545   
PT__Dumping_Site                              -2.925441   
WTP_gate_cnr1                                  0.881545   

                                                             \
                           Average dry season precipitation   
                                  Change in the median in %   
Chimoio_2ML_Reservoir                              1.126932   
Mugica_dam_road                                   -5.900586   
PT1_Pemba__Breidge_Paquite                        -3.362560   
PT_Nicoadala_F5                                   -4.477743   
PT_Revubue_2_Rev_2_01                             -1.507191   
PT__Dumping_Site                                  -3.542630   
WTP_gate_cnr1                                     -1.507191   

                                                         \
                                                          
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                        -11.658801   
Mugica_dam_road                              -13.040792   
PT1_Pemba__Breidge_Paquite                    -8.509459   
PT_Nicoadala_F5                               -9.723633   
PT_Revubue_2_Rev_2_01                         -1.211725   
PT__Dumping_Site                              -9.968265   
WTP_gate_cnr1                                 -1.211725   

                                                         \
                                                          
                           Change in 90-th percentile %   
Chimoio_2ML_Reservoir                          4.855074   
Mugica_dam_road                               -4.254136   
PT1_Pemba__Breidge_Paquite                    -2.097405   
PT_Nicoadala_F5                                0.382411   
PT_Revubue_2_Rev_2_01                         -7.601198   
PT__Dumping_Site                              -3.117568   
WTP_gate_cnr1                                 -7.601198   

                             Extreme rainfall change  \
                            Maximum one day rainfall   
                           Change in the median in %   
Chimoio_2ML_Reservoir                       3.448276   
Mugica_dam_road                             0.000000   
PT1_Pemba__Breidge_Paquite                 -3.448276   
PT_Nicoadala_F5                             2.500000   
PT_Revubue_2_Rev_2_01                       3.5714

### 5-day event

In [30]:
from Functions_Indicators_precipitation import dataframe_n_day_event

In [ ]:
# file of 5 day event
if not os.path.isfile(os.path.join(out_path,'csv_file','pr_5_day_event','pr_5_day_event_'+str(start_year)+'-'+str(stop_year))):
    df_5_day_event=dataframe_n_day_event(precipitation_2021_2060_copy,5) # checker si ca joue. a commencer a 3h20
    path_for_csv = os.path.join(out_path,'csv_file','pr_5_day_event')
    path_to_csv=df_to_csv(df_5_day_event,path_for_csv,'pr_5_day_event_'+str(start_year)+'-'+str(stop_year)+'.csv')
df_5_day_event

Project Chimoio_2ML_Reservoir, scenario ssp126, model ACCESS-CM2
2021
2022
2023


C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:303: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_temp_all_years = df_copy.loc[(project,scenario,model)]


2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
Project Chimoio_2ML_Reservoir, scenario ssp126, model ACCESS-ESM1-5
2021
2022
2023
2024


C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:303: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_temp_all_years = df_copy.loc[(project,scenario,model)]


2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
Project Chimoio_2ML_Reservoir, scenario ssp126, model BCC-CSM2-MR
2021
2022


...

2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
Project PT_Revubue_2_Rev_2_01, scenario ssp245, model NorESM2-LM
2021


C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:303: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_temp_all_years = df_copy.loc[(project,scenario,model)]


2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
Project PT_Revubue_2_Rev_2_01, scenario ssp245, model NorESM2-MM
2021


C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:303: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_temp_all_years = df_copy.loc[(project,scenario,model)]


2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [ ]:
path_for_csv = os.path.join(out_path,'csv_file','pr_5_day_event')
path_to_csv=df_to_csv(df_5_day_event,path_for_csv,'pr_5_day_event_'+str(start_year)+'-'+str(stop_year)+'.csv')

In [ ]:
# future

In [ ]:
# Test for function filldataframe
time = df_future['Date'].tolist()
midx = pd.MultiIndex.from_product([('Chimoio_2ML_Reservoir',),('ssp245',),('ACCESS-CM2' ,), time],names=['Name project','Experiment', 'Model', 'Date'])
name_col = ['bl']#['Precipitation '+str(number_day)+' day event mm']
Variable_dataframe = pd.DataFrame(data = [], 
                            index = midx,
                            columns = name_col)

# Exposure level

In [29]:
# Are project vulnerable ? Exposure level
from Functions_Indicators_precipitation import level_exposure
Exposure=level_exposure(changes_past_future_indicator)
Exposure

NameError: name 'changes_past_future_indicator' is not defined

In [ ]:
changes_past_future_indicator.index.tolist()

In [ ]:
# create empty dataframe
#midx = pd.MultiIndex.from_product([df_years_avg_2041_2060_distribution.index.tolist(),precipitation_2021_2060_copy.index.levels[1].tolist(),models],names=['Name project','Experiment', 'Model'])
cols = pd.MultiIndex.from_product([('Exposure level',),('Incremental air temperature change', 'Extreme temperature increase','Incremental rainfall change','Extreme rainfall change')])
ExposureLevel = pd.DataFrame(data = [],
                            index = precipitation_2021_2060_copy.index.levels[0].tolist(),
                            columns = cols)

In [ ]:
ExposureLevel

In [ ]:
ExposureLevel.index.tolist()

In [ ]:
# test 
~changes_past_future_indicator[abs(changes_past_future_indicator)>2].notnull().values.all() # True, there is at list one NaN
#~changes_past_future_indicator.isnull().values.all() # True, there are no NaN values

In [ ]:
changes_past_future_indicator.columns.levels[0].tolist()

In [ ]:
changes_past_future_indicator[abs(changes_past_future_indicator)>2].notnull().values.any() # check if there if at least one not NaN value

In [ ]:
for name_p in ExposureLevel.index.tolist():
    for climate_variable in changes_past_future_indicator.columns.levels[0].tolist():
        print('For project '+name_p+', climate variable '+climate_variable)
        # select the columns of interest in the list of columns
        col_interest_med= [cols for cols in changes_past_future_indicator.columns.tolist() if climate_variable in cols and changes_past_future_indicator.columns.levels[2][2] in cols]
        col_interest_p10= [cols for cols in changes_past_future_indicator.columns.tolist() if climate_variable in cols and changes_past_future_indicator.columns.levels[2][0] in cols]
        col_interest_p90= [cols for cols in changes_past_future_indicator.columns.tolist() if climate_variable in cols and changes_past_future_indicator.columns.levels[2][1] in cols]

        if (changes_past_future_indicator.loc[(name_p),col_interest_med][abs(changes_past_future_indicator.loc[(name_p),col_interest_med])>20].notnull().values.any()) or (changes_past_future_indicator.loc[(name_p),col_interest_p10][abs(changes_past_future_indicator.loc[(name_p),col_interest_p10])>50].notnull().values.any() or changes_past_future_indicator.loc[(name_p),col_interest_p90][abs(changes_past_future_indicator.loc[(name_p),col_interest_p90])>50].notnull().values.any()):
            # test if there are any True, if any value is over the threshold indicated
            ExposureLevel.loc[name_p,('Exposure level',climate_variable)] = 'High' # attribute value to exposure level
        if (changes_past_future_indicator.loc[(name_p),col_interest_p10][abs(changes_past_future_indicator.loc[(name_p),col_interest_p10])>20].notnull().values.any() or changes_past_future_indicator.loc[(name_p),col_interest_p90][abs(changes_past_future_indicator.loc[(name_p),col_interest_p90])>20].notnull().values.any()):
            # test if there are any True, if any value is over the threshold indicated
            ExposureLevel.loc[name_p,('Exposure level',climate_variable)] = 'Medium' # attribute value to exposure level
        if (changes_past_future_indicator.loc[(name_p),col_interest_p10][abs(changes_past_future_indicator.loc[(name_p),col_interest_p10])<20].notnull().values.any() or changes_past_future_indicator.loc[(name_p),col_interest_p90][abs(changes_past_future_indicator.loc[(name_p),col_interest_p90])<20].notnull().values.any()):
            # test if there are any True, if any value is under the threshold indicated
            ExposureLevel.loc[name_p,('Exposure level',climate_variable)] = 'Low' # attribute value to exposure level

In [ ]:
def exposureColor(series):
    green = 'background-color: lightgreen'
    orange = 'background-color: orange'
    red = 'background-color: red'
    return [red if value == 'High' else orange if value == 'Medium' else green for value in series]

In [ ]:
ExposureLevel.style.apply(exposureColor)

In [ ]:
changes_past_future_indicator.loc[('Chimoio_2ML_Reservoir'),col_interest]

In [ ]:
changes_past_future_indicator.columns.levels[2]

In [ ]:
col_interest= [cols for cols in changes_past_future_indicator.columns.tolist() if 'Incremental rainfall change' in cols and 'Change in the median in %' in cols]

In [ ]:
col_interest

In [ ]:
type(changes_past_future_indicator.columns.tolist()[0])

In [ ]:
#changes_past_future_indicator.columns.levels[0].tolist() # indicators
changes_past_future_indicator.columns.levels[1].tolist()[0] # statistics

In [ ]:
changes_past_future_indicator.columns.levels[0].tolist()